In [19]:
import pandas as pd

In [20]:
dataset = pd.read_csv('processed_perPage_perLine.csv')
dataset = dataset.iloc[:,1:]
dataset.head()

,Data,PageID
0,カスタマイザー タイトル 設定 カスタマイザー サブタイトル 設定 MobiControl ...,0
1,Apple 製品 MobiControl v ManualApple 製品 MobiCont...,1
2,Android 端末 MobiControl v ManualAndroid 設定 モード ...,2
3,Windows PCMobiControl v ManualWindows PC 根本 セキ...,3
4,Windows EmbeddedMobiControl v ManualWindows Em...,4


In [25]:
corpus_text = ' '.join(dataset.Data)
corpus_list = corpus_text.split()
words_list = list(set(corpus_list))
word_ids = list(enumerate(words_list))
# i should be the first vector followed by 受け, Base, 製造元
word_ids

[(0, 'i'),
 (1, '受け'),
 (2, 'Base'),
 (3, '製造元'),
 (4, 'apache'),
 (5, 'LinuxMobiControl'),
 (6, 'wait'),
 (7, 'やすい'),
 (8, 'disable'),
 (9, 'Linker'),
 (10, '替わり'),
 (11, '送出'),
 (12, 'ブラウザプロキシ'),
 (13, 'wifiapenable'),
 (14, "'"),
 (15, '不等号'),
 (16, 'MobiControlWindowsUtility'),
 (17, 'MobiControlMetadata'),
 (18, 'デバッグ'),
 (19, '近接'),
 (20, '経つ'),
 (21, 'ModeProfile'),
 (22, '書式'),
 (23, '文'),
 (24, 'モードアプリカタログパッケージ'),
 (25, '\\<'),
 (26, 'Response'),
 (27, '協定'),
 (28, '辞書'),
 (29, '省け'),
 (30, 'い'),
 (31, 'モダーンアプリケーション'),
 (32, 'ManualPC'),
 (33, 'in'),
 (34, 'ハッキング'),
 (35, 'ローミング'),
 (36, 'エージェントレスインストール'),
 (37, 'ボイス'),
 (38, 'キャッシュ'),
 (39, '定義付け'),
 (40, 'ハンバーガー'),
 (41, 'ManualFirewall'),
 (42, '異なり'),
 (43, 'タイル'),
 (44, '初期'),
 (45, 'y'),
 (46, '開ける'),
 (47, 'とりあえず'),
 (48, 'しよ'),
 (49, '店'),
 (50, 'Service'),
 (51, 'kddi'),
 (52, '働きかけ'),
 (53, 'ISAPI'),
 (54, 'アサインドアクセス'),
 (55, '圧縮'),
 (56, '命名'),
 (57, 'ナビゲーションパネル'),
 (58, 'アプリケーション'),
 (59, 'とりこみ'),
 (60, '並ん'),
 (61

In [26]:
# Vectorizer Just Words -> IDs
def custom_vectorizer(input_list):
    collector = []
    for token in input_list:
        for key, value in word_ids: 
            if value == token:
                # print(value, "->", key)
                collector.append(key)
                break
    return collector

In [27]:
text_vectors=[]
page_ids = []
for index, col in dataset.iterrows():
    text_vectors.append(custom_vectorizer(str(col['Data']).split()))
    page_ids.append(col['PageID'])

vector = list(zip(text_vectors, page_ids))

In [36]:
import pickle

# Not writing on disk because already it is present there
# with open('vectormodel_perpage.pkl', 'wb') as f:
#     pickle.dump(vector ,f)
    
with open('vectormodel_perpage.pkl', 'rb') as f:
    vector = pickle.load(f)
    
len(vector)    

445

In [37]:
import MeCab 
import re
import random
from pathlib import Path

mecab = MeCab.Tagger('-Owakati')

def get_questions_delimiter_ja():
    questions_word = []
    questions_word_file = Path("/home/iftekhar/AI-system/JPBERT/questions_delimiter.txt")
    with open(questions_word_file, encoding='utf-8') as f:
        questions_word_list = f.read().splitlines()
    return questions_word_list

def make_question(split_lines_corpus):
    pages_list = []
    content_list = []
#     mecab_tokenizer = MecabTokenizer()
#     mecab_tokenizer._load_mecab()

    for pages in split_lines_corpus:
        for sentences in pages:
            pages_list.append(sentences[0])
        content_list.append(pages_list)
        pages_list = []

    count = 0
    all_question_list = []
    for pages in content_list:
        question_list = []

        for sent in pages:
            sent = str(sent)

            if re.search(r'\w+', sent):
                #content = mecab_tokenizer.wakati_base_form(sent)
                
                item_list = []
                content = mecab.parse(sent).strip("\n").rstrip()
                item_list = content.split()
                item_list = list(set(item_list))

                #item_list = list(set(content))
                random_item_from_list = random.choice(item_list)
                item_list.remove(random_item_from_list)
                random.shuffle(item_list)

                question_delimiter = random.choice(get_questions_delimiter_ja())
                item_list.append(question_delimiter)

                question = ' '.join(item_list)
                question_list.append([question, count])

        count += 1
        all_question_list.append(question_list)

    labels = []
    text_list = []
    for i in all_question_list:
        for j in i:
            text_list.append(j[0])
            labels.append(j[1])
    question_data = pd.DataFrame(zip(text_list, labels), columns=['Question', 'PageID'])
    question_data = question_data.sample(frac=1).reset_index(drop=True)

    return question_data

def split_lines_corpus(per_page_corpus):
    corpus_list = []
    whole_corpus = []
    n = 100
    word_limit = 10
    for index, row in per_page_corpus.iterrows():
        var = str(row['Data']).split('。')

        for elements in var:
            if len(elements) > n:
                elements = elements.split()
                chunks = [' '.join(elements[i:i + word_limit]) for i in range(0, len(elements), word_limit)]
                for items in chunks:
                    corpus_list.append([items, row['PageID']])
            else:
                corpus_list.append([elements, row['PageID']])
        whole_corpus.append(corpus_list)
        corpus_list = []
    return whole_corpus


In [38]:
split_lines_corpus = split_lines_corpus(dataset)
questions_corpus = make_question(split_lines_corpus)

In [39]:
questions_corpus.head()

,Question,PageID
0,コンテンツ タップ 始まり インストール MobiControl 画面 B なにで,393
1,端末 CSV 指定 シリアル ロード 現れる ファイル Explorer アップ 使うには,387
2,ポート アプリ 型式 ファイル CSV 入力 作成 イン どちら,282
3,せ ダウンロード ドライバ プリントドライバ ユーザー 無効 HTTP パッケージ ついて教...,61
4,グループ ん ば ADDS なけれ メンバー 何ですか,367


In [40]:
sample_size = 1000
questions_samples = questions_corpus.head(int(sample_size))

In [41]:
questions_samples.head()

,Question,PageID
0,コンテンツ タップ 始まり インストール MobiControl 画面 B なにで,393
1,端末 CSV 指定 シリアル ロード 現れる ファイル Explorer アップ 使うには,387
2,ポート アプリ 型式 ファイル CSV 入力 作成 イン どちら,282
3,せ ダウンロード ドライバ プリントドライバ ユーザー 無効 HTTP パッケージ ついて教...,61
4,グループ ん ば ADDS なけれ メンバー 何ですか,367


In [49]:
def get_score_details_record(dataset, questions_samples, vectorizer, vector):
    sample_count = 0
    sum_score = 0
    container = []

    for index, col in questions_samples.iterrows():
        query = str(col['Question'])
        input_vector = list(set(str(query).split()))
        query_vector = custom_vectorizer(input_vector)
        
        rank_list = custom_countvectorizer_ranking(dataset, vector, query, query_vector, vectorizer)

        page_answers = []
        prediction_scores = []
        for items, ids in rank_list:
            # print(items, ids)
            page_answers.append(ids)
            prediction_scores.append(items)

        MRR_score = mean_reciprocal_rank_score(col['PageID'], page_answers)
        sum_score += MRR_score

        container.append([MRR_score, col['PageID'], page_answers, prediction_scores, col['Question']])
        sample_count += 1
        
    result = pd.DataFrame(container, columns=['score', 'actual_answer', 'page_answers', 'prediction_scores',
                                              'query'])
    result.to_csv('performance.csv')
    score = sum_score / sample_count

    return score

def return_match_length(a, b):
    a = set(a)
    b = set(b)
    saver = [list(b - a), list(a - b)]

    flat = [x for sublist in saver for x in sublist]
    matched_length = len(a) + len(b) - len(flat)
    return matched_length

def mean_reciprocal_rank_score(actual_value, predicted_values):
    pos = 0
    val = 0
    for i in predicted_values:
        if i == actual_value and pos == 0:
            val = 1
            break
        elif i == actual_value and pos == 1:
            val = 0.5
            break
        elif i == actual_value and pos == 2:
            val = 0.33
            break
        else:
            val = 0
        pos += 1

    return val

def ranking_result_collection(query, text_collections, ids_list, vec):
    X = vec.transform([str(query)])
    
    rank=[]
    for contents in text_collections:
        Y = vec.transform([contents])
        rank.append(cosine_similarity(X, Y))
    flat = [x for sublist in rank for x in sublist]
    result = sorted(zip(flat, ids_list), reverse=True)[:3]
    return result

def filtered_vector(collectors, vectorizer, dataset):
    text_collections=[]
    ids_list=[]
    for match, ids in collectors:
        text_collections.append(dataset["Data"][ids])
        ids_list.append(ids)
    vec = vectorizer.fit(text_collections)
    return vec, text_collections, ids_list

def collecting_few_results(rank_list):
    collectors = []
    cut=0
    temp=None
    for items, ids in rank_list:
        if items!=temp and cut>1:
            break
        elif items==temp: 
            collectors.append([items,ids])
        else:
            cut+=1
            temp = items
            collectors.append([items,ids])
    return collectors

def rank_list_collection(result_collection):
    matching_frequency = []
    ids_collection = []
    for matches, ids in result_collection:
        matching_frequency.append(matches)
        ids_collection.append(ids)
    rank_list = sorted(zip(matching_frequency,ids_collection), reverse=True)[:50]
    return rank_list

import difflib

def sequence_matcher(sequence1, sequence2):
    matcher = difflib.SequenceMatcher(None, sequence1, sequence2)
    matches = matcher.get_matching_blocks()

    matching_result_collection = []
    for match in matches:
        if len(sequence1[match.a:match.a + match.size]) > 0:
            matching_result_collection.append(sequence1[match.a:match.a + match.size])
    return matching_result_collection   

def sequence_wise_ranking(match_list_collection):
    adder = 0
    score_collection = []
    for i, ids in match_list_collection:
        for items in i:
            if len(items) > 1:
                adder += len(items)
            else:
                adder += 0.5
        score_collection.append([adder,ids])
        adder = 0

    return sorted(score_collection,key=lambda l:l[0], reverse=True)[:30]


def sequence_handler(collectors):
    vector_collections=[]
    ids_list=[]
    for match, ids in collectors:
        vector_collections.append(vector[ids])
    match_list_collection = []
    for items_vector, ids in vector_collections:
        match_list_collection.append([sequence_matcher(items_vector, query_vector),ids])
    sequence_wise_rank = sequence_wise_ranking(match_list_collection)
    return sequence_wise_rank

def custom_countvectorizer_ranking(dataset, vector, query, query_vector, vectorizer):
    result_collection = []
    for words_vector, page_id in vector:    
        #result_collection.append([list(set(words_vector).intersection(set(query_vector))),page_id])
        result_collection.append([return_match_length(words_vector, query_vector), page_id])
        
    rank_list = rank_list_collection(result_collection)
    collectors = collecting_few_results(rank_list)
    sequence_wise_rank = sequence_handler(collectors)
    vec, text_collections, ids_list = filtered_vector(sequence_wise_rank, vectorizer, dataset)
    ranks = ranking_result_collection(query, text_collections, ids_list, vec)
    
    vec, text_collections, ids_list = filtered_vector(collectors, vectorizer, dataset)
    ranks = ranking_result_collection(query, text_collections, ids_list, vec)
    return ranks
    


In [50]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
vectorizer = CountVectorizer()

ranks = get_score_details_record(dataset, questions_samples, vectorizer, vector)
ranks

0.5191899999999998

# Experiments Starts from below


In [45]:
for index, col in questions_samples.iterrows():
    query = str(col['Question'])
    input_vector = list(set(str(query).split()))
    query_vector = custom_vectorizer(input_vector)
    break


In [46]:
result_collection = []
for words_vector, page_id in vector:    
    #result_collection.append([list(set(words_vector).intersection(set(query_vector))),page_id])
    result_collection.append([return_match_length, page_id])
  

In [47]:
result_collection

[[<function __main__.return_match_length(a, b)>, 0],
 [<function __main__.return_match_length(a, b)>, 1],
 [<function __main__.return_match_length(a, b)>, 2],
 [<function __main__.return_match_length(a, b)>, 3],
 [<function __main__.return_match_length(a, b)>, 4],
 [<function __main__.return_match_length(a, b)>, 5],
 [<function __main__.return_match_length(a, b)>, 6],
 [<function __main__.return_match_length(a, b)>, 7],
 [<function __main__.return_match_length(a, b)>, 8],
 [<function __main__.return_match_length(a, b)>, 9],
 [<function __main__.return_match_length(a, b)>, 10],
 [<function __main__.return_match_length(a, b)>, 11],
 [<function __main__.return_match_length(a, b)>, 12],
 [<function __main__.return_match_length(a, b)>, 13],
 [<function __main__.return_match_length(a, b)>, 14],
 [<function __main__.return_match_length(a, b)>, 15],
 [<function __main__.return_match_length(a, b)>, 16],
 [<function __main__.return_match_length(a, b)>, 17],
 [<function __main__.return_match_leng